In [ ]:
import fair
import matplotlib.pyplot as pl
import pandas as pd
import numpy as np
import scipy.stats as st
from tqdm import tqdm
import json
from climateforcing.utils import mkdir_p

In [ ]:
# set up plotting
pl.rcParams['figure.figsize'] = (12/2.54, 9/2.54)
pl.rcParams['font.size'] = 12
pl.rcParams['font.family'] = 'Arial'
pl.rcParams['xtick.direction'] = 'out'
pl.rcParams['ytick.direction'] = 'out'
pl.rcParams['xtick.minor.visible'] = True
pl.rcParams['ytick.major.left'] = True
pl.rcParams['ytick.major.right'] = True
pl.rcParams['ytick.minor.visible'] = True
pl.rcParams['xtick.top'] = True
pl.rcParams['ytick.right'] = True

In [ ]:
ssp_df = pd.read_csv('../data_input/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
years = np.arange(1750, 2111)
years_future = [2015] + list(range(2020,2111,10))

startyear = 1750
first_scenyear = 2015
last_scenyear = 2110
first_row = int(first_scenyear-startyear)
last_row = int(last_scenyear-startyear)

scenarios = ['ssp126']

species = [  # in fair 1.6, order is important
    '|CO2|MAGICC Fossil and Industrial',
    '|CO2|MAGICC AFOLU',
    '|CH4',
    '|N2O',
    '|Sulfur',
    '|CO',
    '|VOC',
    '|NOx',
    '|BC',
    '|OC',
    '|NH3',
    '|CF4',
    '|C2F6',
    '|C6F14',
    '|HFC23',
    '|HFC32',
    '|HFC4310mee',
    '|HFC125',
    '|HFC134a',
    '|HFC143a',
    '|HFC227ea',
    '|HFC245fa',
    '|SF6',
    '|CFC11',
    '|CFC12',
    '|CFC113',
    '|CFC114',
    '|CFC115',
    '|CCl4',
    '|CH3CCl3',
    '|HCFC22',
    '|HCFC141b',
    '|HCFC142b',
    '|Halon1211',
    '|Halon1202',
    '|Halon1301',
    '|Halon2402',
    '|CH3Br',
    '|CH3Cl',
]

# Assume that units coming out of aneris don't change. One day I'll do unit parsing
unit_convert = np.ones(40)
unit_convert[1] = 12/44/1000
unit_convert[2] = 12/44/1000
unit_convert[4] = 28/44/1000
unit_convert[5] = 32/64
unit_convert[8] = 14/46

data_out = {}

for scenario in scenarios:
    data_out[scenario] = np.ones((361, 40)) * np.nan
    data_out[scenario][:,0] = years

    # past emissions from Zeb
    for i, specie in enumerate(species):
        data_out[scenario][:,i+1] = ssp_df.loc[(ssp_df['Region']=='World')&(ssp_df['Scenario']==scenario)&(ssp_df['Variable'].str.endswith(specie)),str(startyear):'2110'].interpolate(axis=1)*unit_convert[i+1]

In [ ]:
pl.plot(data_out[scenario][250:300, 0], data_out[scenario][250:300, 11])

In [ ]:
results_out = {}

In [ ]:
with open('../data_input/fair-ar6-calib/fair-1.6.2-wg3-params.json') as f:
    config_list = json.load(f)

In [ ]:
updated_config = []
for i, cfg in enumerate(config_list):
    updated_config.append({})
    for key, value in cfg.items():
        if isinstance(value, list):
            updated_config[i][key] = np.asarray(value)
        else:
            updated_config[i][key] = value
    updated_config[i]['emissions'] = data_out['ssp126']
    updated_config[i]['diagnostics'] = 'AR6'
    updated_config[i]["efficacy"] = np.ones(45)
    updated_config[i]["gir_carbon_cycle"] = True
    updated_config[i]["temperature_function"] = "Geoffroy"
    updated_config[i]["aerosol_forcing"] = "aerocom+ghan2"
    updated_config[i]["fixPre1850RCP"] = False
    updated_config[i]["scale"][43] = 0.6
    updated_config[i]["F_solar"][270:] = 0

In [ ]:
T = np.zeros((361, len(updated_config)))
F_aer = np.zeros((361, len(updated_config)))
for i in tqdm(range(len(updated_config))):
    thisC, thisF, T[:,i], _, thisOHU, _, thisAF = fair.forward.fair_scm(**updated_config[i])
    F_aer[:,i]=thisF[:,35:41].sum(axis=1)

In [ ]:
hadcrut5_df = pd.read_csv('../data/HadCRUT.5.0.1.0.analysis.summary_series.global.annual.csv')
hadcrut5_df

In [ ]:
mkdir_p('../plots/')

In [ ]:
hadcrut5_df = pd.read_csv('../data_input/gsat/HadCRUT.5.0.1.0.analysis.summary_series.global.annual.csv')

In [ ]:
T_rebase = T-np.mean(T[100:151,:], axis=0)
fig,ax=pl.subplots()
ax.plot(np.arange(1750,2111), np.median(T_rebase, axis=1), color='k', label='FaIR ensemble median')
ax.fill_between(np.arange(1750,2111), np.percentile(T_rebase, 5, axis=1), np.percentile(T_rebase, 95, axis=1), label='FaIR 5-95% range', color="0.7")
ax.plot(hadcrut5_df.loc[0:171,'Time'], hadcrut5_df.loc[0:171,'Anomaly (deg C)']-hadcrut5_df.loc[0:50,'Anomaly (deg C)'].mean(), color='r', label='HadCRUT5')
ax.set_xlim(1980,2100)
ax.set_ylim(0,2.5)
ax.axhline(1.5, ls=':', color='k')
ax.set_title('SSP1-2.6')
ax.legend(loc='lower right')
ax.set_ylabel('Temperature relative to 1850-1900, °C')
pl.figtext(1,0.02,'@chrisroadmap',ha='right', color='0.6')
pl.figtext(0,0.02,'2237 ensemble members, FaIRv1.6.2')
#ax.text(0.99,0.01,'FaIRv1.6.2 2237 constrained ensemble members', transform=ax.transAxes, ha='right')
#ax.text(0.99,)
fig.tight_layout(rect=[0.0,0.05,1,1])
fig.patch.set_facecolor('white')
pl.savefig('../plots/ssp126-default.png')

In [ ]:
fig,ax=pl.subplots()
ax.plot(np.arange(1750,2111), np.median(F_aer, axis=1), color='k', label='FaIR ensemble median')
ax.fill_between(np.arange(1750,2111), np.percentile(F_aer, 5, axis=1), np.percentile(F_aer, 95, axis=1), label='FaIR 5-95% range', color="0.7")
#ax.plot(hadcrut5_df.loc[0:170,'Time'], hadcrut5_df.loc[0:170,'Anomaly (deg C)']-hadcrut5_df.loc[0:50,'Anomaly (deg C)'].mean(), color='r', label='HadCRUT5')
#ax.set_xlim(1980,2100)

In [ ]:
F_aer[269,:].mean()

In [ ]:
accept_prob = st.uniform.rvs(loc=0, scale=1, size=2237, random_state=79)
constraint1 = np.zeros(2237, dtype=bool)
for i in range(2237):
    likelihood = st.norm.pdf(F_aer[269,:][i], loc=-1.52, scale=0.10)/st.norm.pdf(-1.52, loc=-1.52, scale=0.10)
    if likelihood>=accept_prob[i]:
        constraint1[i] = True
    #print(likelihood)
np.sum(constraint1)

In [ ]:
pl.hist(F_aer[269,constraint1])

In [ ]:
F_aer[269,constraint1].mean()

In [ ]:
F_aer[269,constraint1].std()

In [ ]:
accept_prob = st.uniform.rvs(loc=0, scale=1, size=2237, random_state=80)
constraint2 = np.zeros(2237, dtype=bool)
for i in range(2237):
    likelihood = st.norm.pdf(F_aer[269,:][i], loc=-0.40, scale=0.09)/st.norm.pdf(-0.40, loc=-0.40, scale=0.09)
    if likelihood>=accept_prob[i]:
        constraint2[i] = True
    #print(likelihood)
np.sum(constraint2)

In [ ]:
pl.hist(F_aer[269,constraint2])

In [ ]:
F_aer[269,constraint2].std()

In [ ]:
F_aer[269,constraint2].mean()

In [ ]:
fig,ax=pl.subplots()
ax.plot(np.arange(1750,2111), np.median(T_rebase[:,accept_run], axis=1), color='k', label='FaIR ensemble median')
ax.fill_between(np.arange(1750,2111), np.percentile(T_rebase[:,accept_run], 5, axis=1), np.percentile(T_rebase[:,accept_run], 95, axis=1), label='FaIR 5-95% range', color="0.7")
ax.plot(hadcrut5_df.loc[0:171,'Time'], hadcrut5_df.loc[0:171,'Anomaly (deg C)']-hadcrut5_df.loc[0:50,'Anomaly (deg C)'].mean(), color='r', label='HadCRUT5')
ax.set_xlim(1980,2100)
ax.set_ylim(0,2.5)
ax.axhline(1.5, ls=':', color='k')
ax.set_title('SSP1-2.6')
ax.legend(loc='lower right')
ax.set_ylabel('Temperature relative to 1850-1900, °C')
pl.figtext(1,0.02,'@chrisroadmap',ha='right', color='0.6')
pl.figtext(0,0.02,'2237 ensemble members, FaIRv1.6.2')
#ax.text(0.99,0.01,'FaIRv1.6.2 2237 constrained ensemble members', transform=ax.transAxes, ha='right')
#ax.text(0.99,)
fig.tight_layout(rect=[0.0,0.05,1,1])
fig.patch.set_facecolor('white')
#pl.savefig('../plots/ssp126-default.png')

In [ ]:
fig,ax=pl.subplots(figsize=(10,10))
ax.plot(np.arange(1750,2111), np.median(T_rebase[:,constraint1], axis=1), color='b')
ax.fill_between(np.arange(1750,2111), np.percentile(T_rebase[:,constraint1], 5, axis=1), np.percentile(T_rebase[:,constraint1], 95, axis=1), color="b", alpha=0.2, label='strong PD Faer (%.2f $\pm$ %.2f) W m$^{-2}$' % (F_aer[269,constraint1].mean(), F_aer[269,constraint1].std()))
ax.plot(np.arange(1750,2111), np.median(T_rebase[:,constraint2], axis=1), color='g')
ax.fill_between(np.arange(1750,2111), np.percentile(T_rebase[:,constraint2], 5, axis=1), np.percentile(T_rebase[:,constraint2], 95, axis=1), color="g", alpha=0.2, label='weak PD Faer (%.2f $\pm$ %.2f) W m$^{-2}$' % (F_aer[269,constraint2].mean(), F_aer[269,constraint1].std()))
ax.plot(np.arange(1750,2111), np.median(T_rebase, axis=1), color='k')
ax.fill_between(np.arange(1750,2111), np.percentile(T_rebase, 5, axis=1), np.percentile(T_rebase, 95, axis=1), label='AR6 ensemble (%.2f $\pm$ %.2f) W m$^{-2}$' % (F_aer[269,:].mean(), F_aer[269,:].std()), color="k", alpha=0.2)
ax.plot(hadcrut5_df.loc[0:171,'Time'], hadcrut5_df.loc[0:171,'Anomaly (deg C)']-hadcrut5_df.loc[0:50,'Anomaly (deg C)'].mean(), color='r', label='HadCRUT5')
ax.set_xlim(1980,2100)
ax.set_ylim(0,3)
ax.axhline(1.5, ls=':', color='k')
ax.legend(loc='upper left')
ax.set_title('SSP1-2.6')
pl.savefig('../plots/ssp126_three_aer_forcings.png')